In [26]:
import numpy as np
import pandas as pd
from datetime import datetime
import re
from collections import Counter
from wordcloud import WordCloud
import warnings
import emoji

In [27]:
def seperate_date_time(x):
    date = pd.to_datetime(x.split(', ')[0])
    time = x.split(', ')[1].split(' - ')[0]
    return date, time

In [28]:
def createDf(file_data, dt):
    patterns = {
        '12 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    
    df = pd.DataFrame({'UserMessage': user_message, 'DateTime': date_time})
    df[['Date','Time']] = df['DateTime'].apply(lambda x: seperate_date_time(x)).to_list()
    users = []
    messages = []
    for message in df['UserMessage']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['User'] = users
    df['Message'] = messages
    df.drop(columns=['UserMessage', 'DateTime'], inplace=True)
    return df

In [40]:
f = open('chats/WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')
df = createDf(f, '12 Hour')

In [36]:
f = open('chats/WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '24 Hour'))

In [ ]:
f = open('chats/2yearsreport.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '12 Hour'))

In [ ]:
f = open('chats/chat.txt', 'r', encoding='utf-8')
df = createDf(f, '12 Hour')

In [24]:
f = open('chats/WhatsApp Chat with SAY GT.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '12 Hour'))

In [41]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()
df['MonthNum'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['WeekNum'] = df['Date'].dt.week
df['Day'] = df['Date'].dt.day
df['DayName'] = df['Date'].dt.day_name()
df['Hour'] = df['Time'].apply(lambda x: x.split(':')[0])
df['Minute'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[0])

#Uncomment while working with 12 hour file format
df['Meridian'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[1]).str.upper()

df.head()

C:\Users\lenovo\AppData\Local\Temp/ipykernel_11312/236753741.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['WeekNum'] = df['Date'].dt.week


,Date,Time,User,Message,Year,Month,MonthNum,DayOfWeek,WeekNum,Day,DayName,Hour,Minute,Meridian
0,2022-05-09,3:01 PM,group_notification,Messages and calls are end-to-end encrypted. N...,2022,May,5,0,19,9,Monday,3,01,PM
1,2022-05-26,7:21 PM,Clg Divyansh,Robotics ke notes,2022,May,5,3,21,26,Thursday,7,21,PM
2,2022-05-26,7:44 PM,Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...,2022,May,5,3,21,26,Thursday,7,44,PM
3,2022-05-26,7:55 PM,Clg Abbas Amjhera,<Media omitted>,2022,May,5,3,21,26,Thursday,7,55,PM
4,2022-05-26,8:05 PM,+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻,2022,May,5,3,21,26,Thursday,8,05,PM


# Fetch Stats

In [42]:
def fetch_stats(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  num_messages = df.shape[0]
  diff_days = (df.Date.iloc[len(df)-1] - df.Date.iloc[0]).days
  words = []
  for message in df['Message']:
    words.extend(message.split())

  num_media_messages = df[df['Message'] == '<Media omitted>\n'].shape[0]
  return num_messages, len(words), num_media_messages, diff_days

In [43]:
fetch_stats('Overall', df)

(4201, 21507, 0, 144)

# Removed/Left User

In [44]:
def seperate(x, seperator):
    if seperator == 'removed':
        if x.find(seperator) != -1:
            return x.split(seperator)[1]
        else:
            return x.split('left')[0]
    else:
        if x.find(seperator) != -1:
            return x.split(seperator)[1]
        else:
            return x.split('joined')[0]

In [45]:
seperate('Dhakad Svvv removed +91 94240 96950', 'removed')

' +91 94240 96950'

In [46]:
seperate('You left', 'removed')

'You '

In [47]:
seperate('Dhakad Svvv added +91 98937 63178', 'added')

' +91 98937 63178'

In [48]:
seperate('Hrishikesh Svvv joined using this group', 'added')

'Hrishikesh Svvv '

In [49]:
def removed_left(df):
    df = df[df['User']=='group_notification']
    df = df[df['Message'].str.contains('changed|deleted|encrypted|created') == False]
    removedLeft = df[df['Message'].str.contains('removed|left') ==  True]
    addedJoined = df[df['Message'].str.contains('added|joined') == True]
    removedLeft['Status'] = 0
    addedJoined['Status'] = 1
    removedLeft['User'] = removedLeft.Message.apply(lambda x: seperate(x, 'removed'))
    addedJoined['User'] = addedJoined.Message.apply(lambda x: seperate(x, 'added'))
    new_df = pd.concat([removedLeft, addedJoined])
    new_df.drop(columns=['Date','Time', 'Message', 'Year', 'Month', 'MonthNum', 'DayOfWeek', 'WeekNum', 'Day', 'Hour', 'DayName', 'Minute'], inplace=True, axis=1)
    new_df['User'] = new_df['User'].str.lower()
    new_df['User'] = new_df['User'].str.split(', ')
    new_df = new_df.explode('User')
    new_df['User'] = new_df['User'].str.split(' and ')
    new_df = new_df.explode('User')
    new_df['User'] = new_df['User'].str.strip()
    new_df = new_df.groupby('User')['Status'].count().reset_index()
    new_df['Status'] = new_df.Status.apply(lambda x: x%2 == 0)
    return len(new_df[new_df.Status==True])

In [50]:
removed_left(df)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_17328/962002200.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  removedLeft['Status'] = 0
C:\Users\lenovo\AppData\Local\Temp/ipykernel_17328/962002200.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addedJoined['Status'] = 1
C:\Users\lenovo\AppData\Local\Temp/ipykernel_17328/962002200.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

5

# Chatting From

In [51]:
def chat_from(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    unique_years = df['Year'].unique()
    start_year = unique_years[0]
    msg_count = df.groupby(['Date']).count()['Message']
    avg_msg = round(msg_count.mean(),2)
    return start_year, avg_msg

In [52]:
chat_from('Overall', df)

(2022, 34.72)

# Most Talkative

In [53]:
def most_talkative(df):
    df = df[df['User'] != 'group_notification']
    user = df['User'].value_counts()
    username = user.index[0]
    avg_msg = round(user[username]/len(df)*100, 2)
    return username, avg_msg

In [54]:
most_talkative(df)

('Abhsihek Jen Svvv', 13.66)

# Influencer

In [55]:
def influencer(df):
    df = df[df['User'] != 'group_notification']
    new_df = df.groupby(['User'])
    inf_dict = {}
    for name, group in new_df:
        count = 0
        for i in group['Message']:
            if '<Media omitted>' in i:
                count += 1
        inf_dict[name]=count
    name = max(inf_dict, key=inf_dict.get)
    name_df = df[df['User']==name]
    percent = (inf_dict[name]/name_df.shape[0])*100
    return name,round(percent,2), inf_dict[name]

In [56]:
influencer(df)

('Abhsihek Jen Svvv', 9.57, 54)

# Long Winded

In [57]:
def long_winded(df):
    df = df[df['User'] != 'group_notification']
    msg_len = []
    for i in df['Message']:
        msg_len.append(len(i))
    df['Message_len'] = msg_len
    new_df = df.sort_values(by=['Message_len'],ascending=False)
    name = new_df.iloc[0]['User']
    user_df = df[df['User']==name]
    avg_msg_len = int(user_df['Message_len'].mean())
    mean_character = user_df[user_df['Message_len'] > avg_msg_len]
    percentage = round((mean_character.shape[0] / user_df.shape[0])*100, 2)
    return name, avg_msg_len, percentage

In [58]:
long_winded(df)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_17328/894688028.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Message_len'] = msg_len


('Abhsihek Jen Svvv', 20, 33.33)

# Professor

In [59]:
df.sample(15)

,Date,Time,User,Message,Year,Month,MonthNum,DayOfWeek,WeekNum,Day,DayName,Hour,Minute,Meridian
1249,2022-05-30,3:48 pm,Abhsihek Jen Svvv,collage man merji se deta hai,2022,May,5,0,22,30,Monday,3,48,PM
2410,2022-07-08,8:05 pm,Amit Svvv,tu bhe aai ga?,2022,July,7,4,27,8,Friday,8,05,PM
345,2022-02-05,12:15 pm,Abbas,This message was deleted,2022,February,2,5,5,5,Saturday,12,15,PM
3601,2022-08-09,11:52 am,Atman Svvv,Ha kal h,2022,August,8,1,32,9,Tuesday,11,52,AM
1382,2022-01-06,1:10 pm,Charchil Garg,Kesa dost hai yr tu,2022,January,1,3,1,6,Thursday,1,10,PM
3581,2022-06-09,6:04 pm,Aashtha Svvv,October ki bt ho rhj,2022,June,6,3,23,9,Thursday,6,04,PM
2120,2022-07-23,8:22 pm,Anil Svvv,<Media omitted>,2022,July,7,5,29,23,Saturday,8,22,PM
157,2022-04-28,10:36 pm,Abbas,"*IOT Assignment* Keep with you, When required ...",2022,April,4,3,17,28,Thursday,10,36,PM
16,2022-04-25,8:52 pm,Amisha Shukla Svvv,Ok,2022,April,4,0,17,25,Monday,8,52,PM
3035,2022-08-25,1:05 pm,+91 97553 64437,Link kaha h,2022,August,8,3,34,25,Thursday,1,05,PM


# Emoji Lover

In [60]:
def emojized_user(df):
    df=df[df['User']!='group_notification']
    emoji_user = {}
    new_df = df.groupby(['User'])
    for i in df['User'].unique():
        count = 0
        group = new_df.get_group(i)
        for j in group['Message']:
            for k in j:
                if emoji.is_emoji(k):
                    count += 1
        emoji_user[i]=count   
    name = max(emoji_user, key=emoji_user.get)
    total = emoji_user.values()
    percent = round((emoji_user[name]/sum(total))*100, 2)
    return name, percent

In [61]:
emojized_user(df)

('Anmol Svvv', 18.74)

# Early Bird

In [62]:
def early_bird(df, format):
    new_df = df[df['User']!='group_notification']
    if format == '12 Hour':
        new_df = new_df[((new_df['Meridian'] == 'AM') & (pd.to_numeric(new_df['Hour']) > 7)) | (new_df['Meridian'] == 'PM')]
    elif format == '24 Hour':
        new_df = df[pd.to_numeric(df['Hour']) > 7]
    user = new_df['User'].value_counts()
    username = user.index[0]
    avg_msg = round(user[username]/len(new_df)*100, 2)
    return username, avg_msg

In [64]:
early_bird(df, '12 Hour')

('Abhsihek Jen Svvv', 13.53)

In [65]:
early_bird(df, '24 Hour')

('Abhsihek Jen Svvv', 13.5)

# Night Owl

In [66]:
def night_owl(df, format):
    new_df = df[df['User']!='group_notification']
    if format == '12 Hour':
        new_df = new_df[((new_df['Meridian'] == 'AM') & (pd.to_numeric(new_df['Hour']) < 7)) | (new_df['Meridian'] == 'PM')]
    elif format == '24 Hour':
        new_df = new_df[(pd.to_numeric(new_df['Hour']) < 6) | (pd.to_numeric(new_df['Hour']) > 11)]
    user = new_df['User'].value_counts()
    username = user.index[0]
    avg_msg = round(user[username]/len(new_df)*100, 2)
    return username, avg_msg

In [67]:
night_owl(df, '12 Hour')

('Abhsihek Jen Svvv', 13.83)

In [68]:
night_owl(df, '24 Hour')

('Abhsihek Jen Svvv', 11.97)

# Most Shared Links

In [79]:
def get_urls(df, selected_user):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    df = df[df['User'] != 'group_notification']
    urls_list = []
    url = []
    for i in df['Message']:
        o = re.findall('(https://.*)|(http://.*)',i)
        if len(o) != 0:
            urls_list.append(o[0][0].split(' ')[0])
    if len(urls_list) != 0:
        for i in urls_list:
            o = re.search('.*[.](com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)',i)
            if o != None:
                url.append(o.group())
        url_df = pd.DataFrame(Counter(url).most_common(len(Counter(url))), columns=['Urls', 'Count'])
        return url_df
    else:
        return pd.DataFrame()

In [80]:
get_urls(df, 'Overall')

,Urls,Count
0,https://teams.microsoft.com,13
1,https://docs.google.com,6
2,https://forms.gl,5
3,https://chat.whatsapp.com,4
4,https://www.researchgate.net,1
5,https://www.tutorialspoint.com/matlab/matlab_e...,1
6,https://maps.app.goo.gl,1
7,https://bit.ly,1
8,https://www.tcs.com,1
9,https://meet.google.com,1


# Most Shared Emojis

In [81]:
def get_emojis(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    emojis = []
    description = []
    for message in df['Message']:
        all_emoji = emoji.distinct_emoji_list(message)
        emojis.extend([emoji.emojize(is_emoji) for is_emoji in all_emoji])
    emoji_df = pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))), columns=['Emoji', 'Count'])
    for i in emoji_df['Emoji']:
        description.append(emoji.demojize(i))
    emoji_df['Description'] = description
    emoji_df['Description'] = emoji_df['Description'].apply(lambda x: x.strip(':'))
    emoji_df['EmojiDescription'] = emoji_df['Emoji'] + ' - ' + emoji_df['Description']
    return emoji_df

In [82]:
get_emojis('Overall', df)

,Emoji,Count,Description,EmojiDescription
0,😂,147,face_with_tears_of_joy,😂 - face_with_tears_of_joy
1,🤣,95,rolling_on_the_floor_laughing,🤣 - rolling_on_the_floor_laughing
2,😅,57,grinning_face_with_sweat,😅 - grinning_face_with_sweat
3,🙂,34,slightly_smiling_face,🙂 - slightly_smiling_face
4,👍,26,thumbs_up,👍 - thumbs_up
...,...,...,...,...
93,😨,1,fearful_face,😨 - fearful_face
94,💃,1,woman_dancing,💃 - woman_dancing
95,🍇,1,grapes,🍇 - grapes
96,💤,1,zzz,💤 - zzz


# Hourly Timeline

In [67]:
def hourly_timeline(selected_user, df, format):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df['Message'] = [1] * new_df.shape[0]
  if format == '12 Hour':
    new_df['Hour'] = new_df['Hour'].apply(lambda x: ('0'+str(x)) if (len(x)<2) else x )
    new_df['Hour'] = new_df['Meridian'].astype(str) + ' ' + new_df['Hour'].astype(str) 
    new_df = new_df.groupby('Hour').sum().reset_index()
  else:
    new_df = new_df.groupby('Hour').sum().reset_index()
  return new_df

In [68]:
hourly_timeline('Overall', df, '12 Hour')

C:\Users\lenovo\AppData\Local\Temp/ipykernel_11312/1366137675.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Message'] = [1] * new_df.shape[0]
C:\Users\lenovo\AppData\Local\Temp/ipykernel_11312/1366137675.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Hour'] = new_df['Hour'].apply(lambda x: ('0'+str(x)) if (len(x)<2) else x )
C:\Users\lenovo\AppData\Local\Temp/ipykernel_11312/1366137675.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,Hour,Message,Year,MonthNum,DayOfWeek,WeekNum,Day
0,AM 01,14,28308,97,55,395,260
1,AM 02,3,6066,18,10,77,85
2,AM 03,5,10110,31,28,129,128
3,AM 04,1,2022,6,3,22,2
4,AM 07,2,4044,12,6,46,18
5,AM 08,14,28308,97,31,408,322
6,AM 09,44,88968,294,243,1136,462
7,AM 10,49,99078,303,158,1215,804
8,AM 11,103,208266,721,256,2885,1347
9,AM 12,22,44484,141,89,584,497


# Daily Timeline

In [85]:
def daily_timeline(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df = new_df.groupby('Date')['Message'].count().reset_index()
  return new_df

In [86]:
daily_timeline('Overall', df)

,Date,Message
0,2022-01-05,29
1,2022-01-06,172
2,2022-01-07,6
3,2022-01-09,14
4,2022-02-05,33
...,...,...
113,2022-11-09,53
114,2022-12-05,17
115,2022-12-06,29
116,2022-12-08,23


# Weekly Timeline

In [617]:
def weekly_timeline(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df  = df.groupby(['Year','Month','WeekNum'], sort=False)['Message'].count().reset_index()
  week = []
  for i in range(len(new_df)):
    week.append(str(new_df['WeekNum'][i]) + " - " + new_df['Month'][i] + " - " + str(new_df['Year'][i]))
  new_df['Week'] = week
  new_df['Week'] = new_df[['WeekNum', 'Week']].apply(lambda x: "Week 0"+x['Week'] if x['WeekNum']<10 else "Week "+x['Week'], axis=1)
  new_df.sort_values(['WeekNum', 'Month', 'Year'], inplace=True)
  return new_df

In [618]:
weekly_timeline('Overall', df)

,Year,Month,WeekNum,Message,Week
1,2022,January,1,225,Week 01 - January - 2022
2,2022,February,5,42,Week 05 - February - 2022
20,2022,February,6,77,Week 06 - February - 2022
3,2022,March,9,114,Week 09 - March - 2022
21,2022,March,10,112,Week 10 - March - 2022
4,2022,April,14,210,Week 14 - April - 2022
0,2022,April,17,291,Week 17 - April - 2022
5,2022,May,18,65,Week 18 - May - 2022
13,2022,May,19,149,Week 19 - May - 2022
14,2022,May,20,185,Week 20 - May - 2022


In [449]:
weekly_timeline('+91 6239 413 783', df)

,Year,Month,WeekNum,Message,Week
9,2021,January,1,1,Week 2 - January - 2020
0,2020,January,2,2,Week 51 - December - 2020
7,2021,February,5,1,Week 52 - December - 2020
5,2021,March,12,1,Week 17 - May - 2021
6,2021,April,16,4,Week 22 - June - 2021
3,2021,May,17,1,Week 12 - March - 2021
4,2021,June,22,1,Week 16 - April - 2021
13,2021,June,23,1,Week 5 - February - 2021
8,2021,June,24,5,Week 24 - June - 2021
11,2021,September,37,1,Week 1 - January - 2021


# Monthly Timeline

In [632]:
def monthly_timeline(selected_user, df):
    if selected_user != 'Overall':
      df = df[df['User'] == selected_user]
    new_df = df[df['User'] != 'group_notification']
    new_df  = df.groupby(['Year','Month','MonthNum'], sort=False)['Message'].count().reset_index()
    month = []
    for i in range(new_df.shape[0]):
        month.append(str(new_df['MonthNum'][i]) + " - " + new_df['Month'][i] + " - " + str(new_df['Year'][i]))
    new_df['Months'] = month
    new_df['Months'] = new_df[['MonthNum', 'Months']].apply(lambda x: "Month 0"+x['Months'] if x['MonthNum']<10 else "Month "+x['Months'], axis=1)
    new_df.sort_values(['MonthNum', 'Year'], inplace=True)
    return new_df

In [633]:
monthly_timeline('Overall', df)

,Year,Month,MonthNum,Message,Months
1,2022,January,1,225,Month 01 - January - 2022
2,2022,February,2,119,Month 02 - February - 2022
3,2022,March,3,226,Month 03 - March - 2022
0,2022,April,4,501,Month 04 - April - 2022
4,2022,May,5,626,Month 05 - May - 2022
5,2022,June,6,217,Month 06 - June - 2022
6,2022,July,7,302,Month 07 - July - 2022
7,2022,August,8,948,Month 08 - August - 2022
8,2022,September,9,298,Month 09 - September - 2022
9,2022,October,10,220,Month 10 - October - 2022


# Most Busy Day

In [611]:
def most_busy_day(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df = new_df.groupby(['DayName', 'DayOfWeek'], sort=False)['Message'].count().reset_index()
  new_df.sort_values('DayOfWeek', inplace=True)
  new_df['Days'] = new_df['DayOfWeek'].astype(str) + " - " + new_df['DayName']
  return new_df

In [612]:
most_busy_day('Overall', df)

,DayName,DayOfWeek,Message,Days
0,Monday,0,655,0 - Monday
1,Tuesday,1,500,1 - Tuesday
2,Wednesday,2,586,2 - Wednesday
3,Thursday,3,688,3 - Thursday
4,Friday,4,1025,4 - Friday
5,Saturday,5,423,5 - Saturday
6,Sunday,6,252,6 - Sunday


# Most Busy Month

In [626]:
def most_busy_month(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df = new_df.groupby(['MonthNum', 'Month'], sort=False)['Message'].count().reset_index()
  new_df.sort_values('MonthNum', inplace=True)
  new_df['Months'] = new_df['MonthNum'].astype(str) + " - " + new_df['Month']
  new_df['Months'] = new_df[['MonthNum', 'Months']].apply(lambda x: "Month 0"+x['Months'] if x['MonthNum']<10 else "Month "+x['Months'], axis=1)
  return new_df

In [627]:
most_busy_month('Overall', df)
# temp['Months']

,MonthNum,Month,Message,Months
1,1,January,221,Month 01 - January
2,2,February,119,Month 02 - February
3,3,March,223,Month 03 - March
0,4,April,482,Month 04 - April
4,5,May,614,Month 05 - May
5,6,June,216,Month 06 - June
6,7,July,299,Month 07 - July
7,8,August,924,Month 08 - August
8,9,September,297,Month 09 - September
9,10,October,218,Month 10 - October


# User Chat Percentage

In [132]:
def user_chat_percentage(df):
    df = df[df['User'] != 'group_notification']
    user = df['User'].value_counts()
    new_df = pd.DataFrame ({ 'User': user.index, 'Message': user})
    new_df['Percentage'] = new_df['Message'].apply(lambda x: round(x/len(df)*100, 2))
    new_df['User'] = new_df['Percentage'].astype(str) + "% - " + new_df['User']
    new_df.drop('Message', axis=1, inplace=True)
    new_df.reset_index(drop=True,inplace=True)
    return new_df

In [133]:
user_chat_percentage(df)

,User,Percentage
0,15.07% - Clg Ashvin birla,15.07
1,11.49% - Clg Abhishek Jain,11.49
2,9.11% - Clg Kushagra Ahire,9.11
3,8.55% - Clg Amit Pandey,8.55
4,8.27% - Clg Abbas Amjhera,8.27
5,4.13% - Clg Divyansh,4.13
6,4.13% - Clg Charchil,4.13
7,3.92% - Clg Anmol Joshi,3.92
8,3.36% - +91 96309 40258,3.36
9,3.08% - +91 72409 15291,3.08


# Most Common Word

In [11]:
def most_common_words(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df = new_df[~new_df['Message'].str.contains('<Media omitted>')]
  f = open('stop_hinglish.txt')
  stop_words = f.read()

  words = []

  for message in new_df['Message']:
    message = re.sub('[^A-Za-z\s]','',message)
    if message == '':
        pass
    else:
        for word in message.lower().split():
            if word not in stop_words:
                words.append(word)
  
  most_common_df = pd.DataFrame(Counter(words).most_common(20), columns=['Message', 'Count'])
  most_common_df.sort_values('Count')
  return most_common_df

In [12]:
most_common_words('Overall', df)

,Message,Count
0,message,70
1,deleted,67
2,tcs,43
3,form,29
4,jain,25
5,mail,20
6,drive,20
7,apply,19
8,digital,19
9,option,17


# Word Cloud

In [274]:
def create_wordcloud(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df = new_df[new_df['Message'] != '<Media omitted>\n']
  
  f = open('stop_hinglish.txt')
  stop_words = f.read()

  def remove_stop_words(message):
    y = []
    for word in message.lower().split():
      if word not in stop_words:
        y.append(word)
    return " ".join(y)

  wc = WordCloud(width=500, height=500, min_font_size=10)
  new_df['Message'] = new_df['Message'].apply(remove_stop_words)
  df_wc = wc.generate(new_df['Message'].str.cat(sep=" "))
  return df_wc

In [275]:
create_wordcloud('Overall', df)

In [406]:
pd.to_datetime('25-04-2022')

Timestamp('2022-04-25 00:00:00')

# Sentimental Analysis

# Generating Words.xlsx

In [21]:
def remove_emojis(data):
    return emoji.demojize(data)

In [6]:
new_df = df[df['User']!='group_notification']
new_df = new_df[new_df['Message']!='<Media omitted> ']
new_df = new_df[new_df['Message']!='This message was deleted'] 
# new_df['Message'] = new_df['Message'].apply(remove_emojis)

In [7]:
new_df

,Date,Time,User,Message,Year,Month,MonthNum,DayOfWeek,WeekNum,Day,DayName,Hour,Minute,Meridian
1,2022-05-26,7:21 PM,Clg Divyansh,Robotics ke notes,2022,May,5,3,21,26,Thursday,7,21,PM
2,2022-05-26,7:44 PM,Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...,2022,May,5,3,21,26,Thursday,7,44,PM
4,2022-05-26,8:05 PM,+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻,2022,May,5,3,21,26,Thursday,8,05,PM
5,2022-05-26,8:05 PM,Clg Kushagra Ahire,mail pe dekh,2022,May,5,3,21,26,Thursday,8,05,PM
6,2022-05-26,8:05 PM,Clg Kushagra Ahire,mam ne bheje hai bahut saare notes,2022,May,5,3,21,26,Thursday,8,05,PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,2022-08-18,2:00 PM,Clg Divyansh,Yess,2022,August,8,3,33,18,Thursday,2,00,PM
1452,2022-08-18,2:01 PM,Clg Charchil,🤚,2022,August,8,3,33,18,Thursday,2,01,PM
1453,2022-08-18,2:01 PM,+91 87705 37687,✋🏻,2022,August,8,3,33,18,Thursday,2,01,PM
1454,2022-08-18,2:15 PM,Clg Atman,🙋‍♂️,2022,August,8,3,33,18,Thursday,2,15,PM


In [8]:
import stemmer

In [19]:
def text_transformation(words_list):
    corpus = []
    myStemmer = stemmer.Stemmer()
    for item in words_list:
        new_item = item.lower()
        new_item = re.sub('[^a-z]',' ', str(new_item))
        if ('https://' or 'http://') in new_item:
            pass
        else:
            new_item = new_item.split()
            for i in new_item:
                i = myStemmer.stemWord(i)
                if len(i)>1:
                    corpus.append(re.sub('[^a-z]','', i))
    return list(set(corpus))

In [20]:
words = text_transformation(new_df['Message'])
words.sort()
words

['ab',
 'abas',
 'abe',
 'abhi',
 'abhisbek',
 'abhishek',
 'about',
 'above',
 'abr',
 'abstract',
 'abto',
 'acha',
 'ache',
 'achi',
 'achr',
 'achut',
 'achyut',
 'acording',
 'acordingly',
 'acropolis',
 'activity',
 'ad',
 'adhar',
 'adres',
 'advance',
 'advanced',
 'ae',
 'aega',
 'afe',
 'after',
 'agar',
 'agaya',
 'age',
 'agle',
 'agre',
 'agya',
 'ai',
 'aise',
 'aishvarya',
 'aishwariyanhi',
 'aj',
 'aja',
 'ajao',
 'ajata',
 'ajayega',
 'ajenge',
 'akhain',
 'akhiri',
 'al',
 'alag',
 'alawa',
 'alg',
 'algebra',
 'alow',
 'also',
 'alvid',
 'alvida',
 'am',
 'amazon',
 'ambasador',
 'ameting',
 'amit',
 'ana',
 'and',
 'andar',
 'android',
 'ane',
 'ang',
 'ankit',
 'ans',
 'anshul',
 'any',
 'anybody',
 'ao',
 'aoc',
 'ap',
 'apan',
 'apear',
 'apke',
 'apko',
 'aplication',
 'aplied',
 'aply',
 'apn',
 'apna',
 'apne',
 'apni',
 'aproval',
 'apti',
 'aptitude',
 'ar',
 'ara',
 'aray',
 'arays',
 'archana',
 'architecture',
 'are',
 'arey',
 'art',
 'as',
 'asa',
 'asa

In [21]:
words = pd.DataFrame(words, columns=['Word'])
words['Label'] = 1
words.to_excel("Temp.xlsx", index=False)

In [12]:
list1 = text_transformation(new_df['Message'])

In [18]:
list2 = text_transformation(new_df['Message'])

In [23]:
list3 = text_transformation(new_df['Message'])

In [28]:
list4 = text_transformation(new_df['Message'])

In [33]:
words = list(set(list1 + list2 + list3 + list4))
words.sort()

In [2]:
words = pd.DataFrame(words, columns=['Word'])
words['Label'] = 1
words.to_excel("Words.xlsx", index=False)

NameError: name 'words' is not defined

In [106]:
words_df = pd.read_excel('Words.xlsx')
words_df.sample(5)

,Word,Label
3463,kitne,1.0
521,badhaiya,1.0
3683,layga,1.0
5925,squad,1.0
5271,remaining,1.0


In [107]:
words_df.isnull().sum()

Word     1
Label    0
dtype: int64

In [108]:
words_df.dropna(inplace=True)

In [109]:
X = words_df['Word']
y = words_df['Label']

In [110]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [111]:
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [112]:
corpus = []
# for word in X_train:
for word in X:
    corpus.append(str(word))
corpus

['ab',
 'abae',
 'abas',
 'abcd',
 'abd',
 'abdul',
 'abe',
 'abey',
 'abhai',
 'abhar',
 'abhe',
 'abhi',
 'abhinandan',
 'abhinayati',
 'abhisbek',
 'abhisek',
 'abhisex',
 'abhishek',
 'abhishekak',
 'abhishkeh',
 'abi',
 'able',
 'ablove',
 'about',
 'above',
 'abr',
 'absent',
 'abshiek',
 'absolutely',
 'abstract',
 'abstracting',
 'abstraction',
 'abto',
 'ac',
 'acad',
 'academic',
 'academics',
 'academy',
 'ace',
 'acelerating',
 'aceleration',
 'acelerator',
 'acept',
 'aceptable',
 'acepted',
 'acepting',
 'aceptional',
 'aces',
 'acfb',
 'acha',
 'achcha',
 'ache',
 'achi',
 'achr',
 'achut',
 'achuyut',
 'achyut',
 'acio',
 'acording',
 'acordingly',
 'acount',
 'acountability',
 'acquisition',
 'acropolis',
 'acros',
 'acses',
 'act',
 'action',
 'activate',
 'active',
 'activities',
 'activity',
 'ad',
 'ada',
 'adapters',
 'adat',
 'aded',
 'ader',
 'adha',
 'adhar',
 'adhe',
 'adhi',
 'adhwahvztyi',
 'adi',
 'ading',
 'aditya',
 'adjust',
 'admi',
 'admin',
 'administ

In [113]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2))
X = cv.fit_transform(corpus)
# y = y_train

In [114]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [115]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report

In [116]:
rfc = RandomForestClassifier()
rfc.fit(X,y)

RandomForestClassifier()

In [117]:
# test_corpus = []
# for word in X_test:
#     test_corpus.append(str(word))
# test_corpus

In [118]:
# X_test = cv.transform(X_test)

In [119]:
# predictions = rfc.predict(X_test)

In [120]:
# accuracy_score(y_test, predictions)

In [148]:
prediction = rfc.predict(cv.transform(['kala', 'teri', 'bc'])).tolist()
total = len(prediction)
negative = total - prediction.count(1.0)
positive = total - negative
negative_per = round(negative / total * 100, 2)
positive_per = round(positive / total * 100, 2)
total, negative, positive, negative_per, positive_per

(3, 1, 2, 33.33, 66.67)

In [124]:
words_df[words_df['Word'] == 'bc']

,Word,Label
615,bc,0.0


In [123]:
words_df[words_df['Word'] == 'teri']

,Word,Label
6176,teri,1.0


In [122]:
words_df[words_df['Word'] == 'kala']

,Word,Label
3240,kala,1.0


In [8]:
import pickle
with open('semtimental_analysis_model.pickle', 'wb') as f:
    pickle.dump(rfc, f)

NameError: name 'rfc' is not defined

In [163]:
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(cv, f)

In [9]:
model = pickle.load(open("semtimental_analysis_model.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

In [10]:
new_df = df[df['Message']!='<Media omitted>\n']
new_df = new_df[new_df['Message']!='This message was deleted']
new_df['Message'] = new_df['Message'].apply(remove_emojis)
words = text_transformation(new_df['Message'])

In [11]:
words

['pehle',
 'kar',
 'joke',
 'op',
 'instructions',
 'usko',
 'unko',
 'ne',
 'kra',
 'pdega',
 'smjhayenge',
 'bura',
 'hnh',
 'reviews',
 'kushgra',
 'bd',
 'dear',
 'xriteria',
 'count',
 'ye',
 'glas',
 'coleges',
 'multiple',
 'day',
 'reply',
 'flushed',
 'thread',
 'policy',
 'story',
 'raja',
 'calm',
 'arays',
 'btcs',
 'mzjmnju',
 'dig',
 'kardi',
 'pr',
 'punya',
 'pouchne',
 'nho',
 'eyebrow',
 'hoja',
 'hori',
 'bethke',
 'nikalne',
 'claping',
 'board',
 'program',
 'que',
 'pasport',
 'pehel',
 'hmare',
 'aega',
 'behind',
 'zindagi',
 'fake',
 'date',
 'informed',
 'can',
 'itne',
 'numerical',
 'sections',
 'babe',
 'end',
 'de',
 'ly',
 'filed',
 'wk',
 'sab',
 'fir',
 'cases',
 'sataegi',
 'utri',
 'idhar',
 'pasout',
 'complaint',
 'gren',
 'esa',
 'maybe',
 'him',
 'load',
 'dcla',
 'uspe',
 'jisne',
 'beb',
 'package',
 'rehta',
 'batches',
 'others',
 'learn',
 'make',
 'destroyed',
 'pm',
 'join',
 'usmai',
 'pd',
 'jagat',
 'soft',
 'rhe',
 'jise',
 'ghar',
 'ji

In [15]:
prediction = model.predict(vectorizer.transform(words)).tolist()

In [18]:
def sentimental_analysis(df):
    new_df = df[df['Message']!='<Media omitted>\n']
    new_df = new_df[new_df['Message']!='This message was deleted']
    new_df['Message'] = new_df['Message'].apply(remove_emojis)
    words = text_transformation(new_df['Message'])
    prediction = model.predict(vectorizer.transform(words)).tolist()
    total = len(prediction)
    negative = total - prediction.count(1.0)
    positive = total - negative
    negative_per = round(negative / total * 100, 2)
    positive_per = round(positive / total * 100, 2)
    return negative_per, positive_per

In [19]:
sentimental_analysis(df)

(13.02, 86.98)